In [ ]:
from swagger_parser import SwaggerParser

parser = SwaggerParser(swagger_path='swagger.json')  # Init with file

# Get an example of dict for the definition Foo
parser.definitions_example.get('Pet')

In [2]:
import json

with open("swagger.json","r") as infile:
    swagger = json.load(infile)

In [3]:
from faker import Faker
from chance import chance
import random
fake = Faker()


In [168]:
property_generators = {
    "date-time": lambda: fake.date_time(),
    "username":lambda: chance.email(),
    "email": lambda: chance.email(),
    "id":lambda: fake.uuid4(),
    "quantity":lambda: random.randint(0,100),
    "int":lambda: random.randint(0,100),
    "integer":lambda: random.randint(0,100),
    "bool": lambda: chance.boolean(),
    "boolean": lambda: chance.boolean(),
    "firstName": lambda: fake.name().split(" ")[0],
    "lastName": lambda: fake.name().split(" ")[-1],
    "password": lambda: fake.password(),
    "phone": lambda: fake.phone_number(),
    "url": lambda: chance.url(),
    "photoUrls": lambda: chance.url(),
    "status": lambda: "dead" # this is malformed
}


def generate_from_schema(schema_string, name):
    return lambda: list(generate(schema_string.split("/")[-1]))[0]

def generate_array_from_schema(schema_string, name):
    return lambda: [generate_from_schema(schema_string, name)() for x in range(random.randint(0,10))]

def generate_from_enum(enum):
    return lambda: random.choice(enum)

def generate_array_from_enum(enum):
    return lambda: [generate_from_enum(enum)() for x in range(random.randint(0,10))]

def generate_prop_from_name(name):
    return lambda: property_generators[name]() if name in property_generators else name + str(random.randint(0,10))

def generate_array_from_name(name):
    return lambda: [generate_prop_from_name(name)() for x in range(random.randint(0,10))]

def generate_prop_from_type(typ):
    return lambda:  property_generators[typ]() 

def generate_array_from_type(typ):
    return lambda:  [property_generators[typ]()]

def transform_instance_from_schema_to_nested(generator, prop_name):
    generated = generator()
    keys = list(generated.keys())
    for k in keys:
        generated[prop_name + "_" + k] = generated[k]
        del generated[k]
    return generated
    
def get_parameter_generator_from_definition_property(prop_name, prop_val):
    if "$ref" in prop_val:
        generated = generate_from_schema(prop_val["$ref"], prop_name)
        return lambda: transform_instance_from_schema_to_nested(generated, prop_name)
    elif prop_val["type"] == "array" and "enum" in prop_val["items"]:
        return generate_array_from_enum(prop_val["items"]["enum"])
    elif prop_val["type"] == "array" and "type" in prop_val["items"] and prop_val["items"]["type"] == "string":
        return generate_array_from_name(prop_name)
    elif prop_val["type"] == "array" and "type" in prop_val["items"] and prop_val["items"]["type"] == "integer":
        return generate_array_from_type(prop_val["items"]["type"])
    elif prop_val["type"] == "array" and "$ref" in prop_val["items"]:
        return generate_array_from_schema(prop_val["items"]["$ref"], prop_name)
    elif prop_val["type"] == "string" and "enum" in prop_val:
        return generate_from_enum(prop_val["enum"])
    elif prop_val["type"] == "string":
        return generate_prop_from_name(prop_name)
    elif prop_val["type"] in ["int", "integer", "bool", "boolean"]:
        return generate_prop_from_type(prop_val["type"])
    else:
        print(prop_val)
        raise Exception(prop_name)

def get_command_parameter_generator_from_method_param(param):
    if "schema" in param:
        if "$ref" in param["schema"]:
            return generate_from_schema(param["schema"]["$ref"], param["name"])
        elif param["schema"]["type"] == "array":
            return generate_array_from_schema(param["schema"]["items"]["$ref"], param["name"])
        else: 
            raise Exception()
    elif "$ref" in param:
        return generate_from_schema(param["$ref"], param["name"])
    elif param["type"] == "array" and "enum" in param["items"]:
        return generate_array_from_enum(param["items"]["enum"])
    elif param["type"] == "array" and param["items"]["type"] == "string":
        return generate_array_from_name(param["name"])
    elif param["type"] == "string" and "enum" in param:
        return generate_from_enum(param["enum"])
    elif param["type"] == "string":
        return generate_prop_from_name(param["name"])
    elif param["type"] == "int" or param["type"] == "integer":
        return generate_prop_from_type(param["type"])
    elif param["type"] == "file":
        return None
    else:
        raise Exception(param)

In [169]:
object_generators = {}

def get_generator_for_definition(definition_name):
    definition = swagger['definitions'][definition_name] 
    def object_generator():
        instance = {}       
        # iterate through properties 
        for prop_name in definition['properties']:
            val_gen = get_parameter_generator_from_definition_property(prop_name, definition['properties'][prop_name])
            instance[prop_name] = val_gen() if val_gen is not None else None
        yield instance
    return object_generator
    
for definition_name in swagger["definitions"]:
    object_generators[definition_name] = get_generator_for_definition(definition_name)

def generate(obj_name):
    if obj_name not in object_generators:
        raise Exception("Object " + obj_name + " is unknown")
    return object_generators[obj_name]()

list(generate("Pet"))

[{'id': 32,
  'category': {'category_id': 92, 'category_name': 'foo'},
  'name': 'foo',
  'photoUrls': ['pawhinow.org/najuhu/latop/holulu/mudso',
   'zegemem.gov/mupo/dulubo/jobnuble/coserlu/jifla',
   'solop.edu/poce/bogehdi/tufceca/jefge/duwuba',
   'cogow.io/pocvov/rodmudi/lazzewkum/vecove/guwo/ruhaze',
   'wafla.io/hihtubfap/vezocpo/fessa/tatde/rujev',
   'simadmu.org/doclogna/rowsudkoc/vijbicreh/guzejpi/wewomzi/bahpanef',
   'zamu.edu/mukuhnos/hevhep/lizsu/kablopzuw/mariw/tifguv',
   'tipgice.org/hesegeh/tekvidzon/cihi/jokciha/zurani'],
  'tags': [{'id': 1, 'name': 'foo'},
   {'id': 15, 'name': 'foo'},
   {'id': 10, 'name': 'foo'},
   {'id': 85, 'name': 'foo'},
   {'id': 93, 'name': 'foo'},
   {'id': 94, 'name': 'foo'},
   {'id': 60, 'name': 'foo'},
   {'id': 66, 'name': 'foo'}],
  'status': 'available'}]

In [170]:
commands = {}
for path_name in swagger["paths"]:
    path = swagger["paths"][path_name]
    for method in path:
        command = path[method]["summary"] if len(path[method]["summary"]) > 0 else path[method]["description"]
        for param in path[method]["parameters"]:
            if param["in"] == "header":
                continue
            commands[command] = (param["name"],get_command_parameter_generator_from_method_param(param))
commands

{'Add a new pet to the store': ('body',
  <function __main__.generate_from_schema.<locals>.<lambda>()>),
 'Update an existing pet': ('body',
  <function __main__.generate_from_schema.<locals>.<lambda>()>),
 'Finds Pets by status': ('status',
  <function __main__.generate_array_from_enum.<locals>.<lambda>()>),
 'Finds Pets by tags': ('tags',
  <function __main__.generate_array_from_name.<locals>.<lambda>()>),
 'Find pet by ID': ('petId',
  <function __main__.generate_prop_from_type.<locals>.<lambda>()>),
 'Updates a pet in the store with form data': ('status',
  <function __main__.generate_prop_from_name.<locals>.<lambda>()>),
 'Deletes a pet': ('petId',
  <function __main__.generate_prop_from_type.<locals>.<lambda>()>),
 'uploads an image': ('file', None),
 'Place an order for a pet': ('body',
  <function __main__.generate_from_schema.<locals>.<lambda>()>),
 'Find purchase order by ID': ('orderId',
  <function __main__.generate_prop_from_type.<locals>.<lambda>()>),
 'Delete purchase or

In [171]:
for command in commands:
    if commands[command][1] is not None:
        invoked = commands[command][1]()
        if type(invoked) is list and any([x is None for x in invoked]): 
            print(command)
            print(invoked)

In [172]:
import re
from datetime import datetime
import time

def random_prop(obj):
    return str(obj[random.choice(list(obj.keys()))])
    

def construct_sentence_from_generated_method_parameters(parameter_name, parameter_val, sentence):
    if type(parameter_val) is list:
        parameter_val = [p if p is not None else "" for p in parameter_val]
        pstring = ",".join([random_prop(pv) if type(pv) is dict else pv for pv in parameter_val])
        sentence += parameter_name + " " + pstring + ","
    elif type(parameter_val) is str or type(parameter_val) is bool or type(parameter_val) is int:
        sentence += ", " + parameter_name + " " + str(parameter_val)
    else:        
        if parameter_val is None:
            return sentence
        for k in parameter_val:
            if type(parameter_val[k]) is dict:
                sentence += k + " " + str(random_prop(parameter_val[k])) + ", "
            elif type(parameter_val[k]) is list:
                pstring = ",".join([pv["name"] if type(pv) is dict else pv for pv in parameter_val[k]])
                sentence += k + " " + pstring + ","
            else:
                sentence += k + " " + str(parameter_val[k]) + ", "
    return sentence

for command in commands:
    # generate an utterance that contains all of the command and all of the parameters
    parameter_name = commands[command][0]
    parameter_val = commands[command][1]() if commands[command][1] is not None else None
    sentence = construct_sentence_from_generated_method_parameters(parameter_name, parameter_val, command + " with ")    
    print(sentence)

Add a new pet to the store with id 87, category 83, name foo, photoUrls ,tags foo,foo,foo,foo,status available, 
Update an existing pet with id 40, category foo, name foo, photoUrls bucu.edu/vuthulca/vijzem/benodib/puzni/fife/vofotet,sosep.org/varupro/kulas/cetub/cojpu,tags foo,status sold, 
Finds Pets by status with status sold,pending,available,pending,available,available,sold,
Finds Pets by tags with tags tags6,tags4,tags2,tags5,tags6,tags5,tags3,tags2,
Find pet by ID with , petId 100
Updates a pet in the store with form data with , status dead
Deletes a pet with , petId 30
uploads an image with 
Place an order for a pet with id 26, petId 18, quantity 40, shipDate shipDate9, status placed, complete False, 
Find purchase order by ID with , orderId 8
Delete purchase order by ID with , orderId 59
Create user with id 43, username pevoge@jilihi.org, firstName Peter, lastName Meza, email hevmovas@fibegde.co.uk, password G(LE7Ivg!J, phone 001-596-031-1071x456, userStatus 15, 
Creates list 

In [ ]:
sentence